In [163]:
#IMPORTS
import re
import cobra
import numpy as np
import pandas as pd
from utils.importExcelModel import *

In [164]:
#LOAD MODEL
model_path = 'models'
model_filename = 'ecoli_isobutanol.xlsx'
model = import_excel_model('/'.join([model_path, model_filename]))
model.summary()

Index(['Abbreviation', 'Charged formula', 'Description', 'Compartment',
       'Charge'],
      dtype='object')
Loading metabolites...
Loading reactions...


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.003649,0,0.00%
cl_e,EX_cl_e,0.003649,0,0.00%
cobalt2_e,EX_cobalt2_e,0.002433,0,0.00%
cu2_e,EX_cu2_e,0.002433,0,0.00%
fe2_e,EX_fe2_e,0.01129,0,0.00%
glc__D_e,EX_glc___D_e,8,6,98.78%
k_e,EX_k_e,0.1368,0,0.00%
mg2_e,EX_mg2_e,0.006082,0,0.00%
mn2_e,EX_mn2_e,0.002433,0,0.00%
mobd_e,EX_mobd_e,0.002433,0,0.00%


In [165]:
#conversion to new fromate version:
def old_version_model_to_new(model):
    modeltmp=model.copy()
    for metabolite in modeltmp.metabolites:
        metabolite.id = re.sub('\[c\]$',r'_c',metabolite.id)
        metabolite.id = re.sub('\[p\]$',r'_p',metabolite.id)
        metabolite.id = re.sub('\[e\]$',r'_e',metabolite.id)
    print("Parsing metabolites...")
    modeltmp.repair()
    for rxn in modeltmp.reactions:
        rxn.id = re.sub('\(p\)$',r'_p',rxn.id)
        rxn.id = re.sub('\(c\)$',r'_c',rxn.id)
        rxn.id = re.sub('\(e\)$',r'_e',rxn.id)
        # To solve possible problems in changing names
    modeltmp.repair()
    print("Parsing reactions...")
    return(modeltmp)

model = old_version_model_to_new(model)
model.summary()

Read LP format model from file /tmp/tmpg7b5g57s.lp
Reading time = 0.01 seconds
: 2032 rows, 5898 columns, 23086 nonzeros
Parsing metabolites...
Parsing reactions...


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.003649,0,0.00%
cl_e,EX_cl_e,0.003649,0,0.00%
cobalt2_e,EX_cobalt2_e,0.002433,0,0.00%
cu2_e,EX_cu2_e,0.002433,0,0.00%
fe2_e,EX_fe2_e,0.01129,0,0.00%
glc__D_e,EX_glc___D_e,8,6,98.78%
k_e,EX_k_e,0.1368,0,0.00%
mg2_e,EX_mg2_e,0.006082,0,0.00%
mn2_e,EX_mn2_e,0.002433,0,0.00%
mobd_e,EX_mobd_e,0.002433,0,0.00%


In [166]:
print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')

2949 reactions
2032 metabolites
1459 genes


In [167]:
model.compartments

{'e': '', 'c': '', 'p': ''}

In [168]:
#MAKE THE CHANGES NEEDED TO PRODUCE ISOBUTANOL
model.reactions.get_by_id('EX_glc___D_e').bounds = (0,1000)
model.reactions.get_by_id('EX_ac_e').bounds = (-12.8, 1000)
model.reactions.get_by_id('POR5').bounds = (0, 0) #INCLUDE KO AFTER IN GCFRONT RESULTS
model.reactions.get_by_id('ME2').bounds = (3.5, 1000)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,12.8,2,99.15%
ca2_e,EX_ca2_e,0.001352,0,0.00%
cl_e,EX_cl_e,0.001352,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0009015,0,0.00%
cu2_e,EX_cu2_e,0.0009015,0,0.00%
fe2_e,EX_fe2_e,0.004184,0,0.00%
h_e,EX_h_e,10.15,0,0.00%
k_e,EX_k_e,0.0507,0,0.00%
mg2_e,EX_mg2_e,0.002254,0,0.00%
mn2_e,EX_mn2_e,0.0009015,0,0.00%


In [169]:
cobra.io.write_sbml_model(model, model_filename.replace('.xlsx','.xml'))

In [170]:
final_model = cobra.io.read_sbml_model(model_filename.replace('.xlsx','.xml'))
final_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,12.8,2,99.15%
ca2_e,EX_ca2_e,0.001352,0,0.00%
cl_e,EX_cl_e,0.001352,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0009015,0,0.00%
cu2_e,EX_cu2_e,0.0009015,0,0.00%
fe2_e,EX_fe2_e,0.004184,0,0.00%
h_e,EX_h_e,10.15,0,0.00%
k_e,EX_k_e,0.0507,0,0.00%
mg2_e,EX_mg2_e,0.002254,0,0.00%
mn2_e,EX_mn2_e,0.0009015,0,0.00%


In [160]:
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
fseof = FSEOF(final_model)
fseof_result = fseof.run(target=final_model.reactions.EX_btd_RR_e)

In [161]:
fseof_result

,1,2,3,4,5,6,7,8,9,10
12DGR161tipp,0.001970,0.001778,0.001587,0.001395,0.001203,0.001011,0.000819,0.000627,0.000436,0.000244
ACS,12.915362,12.904129,12.892897,12.881664,12.870431,12.859198,12.847966,12.836733,12.825500,12.814268
ACt2rpp,12.800000,12.800000,12.800000,12.800000,12.800000,12.800000,12.800000,12.800000,12.800000,12.800000
ADK1,13.752776,13.660005,13.567234,13.474463,13.381692,13.288921,13.196150,13.103379,13.010607,12.917836
ADK3,-0.171208,-0.154537,-0.137867,-0.121197,-0.104526,-0.087856,-0.071186,-0.054515,-0.037845,-0.021174
ADPT,0.000120,0.000109,0.000097,0.000085,0.000074,0.000062,0.000050,0.000038,0.000027,0.000015
ALAR,0.601427,0.542867,0.484306,0.425746,0.367185,0.308625,0.250064,0.191504,0.132943,0.074383
ALATA_L,-0.747631,-0.674835,-0.602039,-0.529242,-0.456446,-0.383650,-0.310854,-0.238057,-0.165261,-0.092465
ALCD19,0.226946,0.204849,0.182751,0.160654,0.138556,0.116458,0.094361,0.072263,0.050166,0.028068
DAAD,0.590172,0.532707,0.475243,0.417778,0.360313,0.302849,0.245384,0.187920,0.130455,0.072991


In [162]:
data = fseof_result.data_frame
data['flux_difference'] = [row[10]-row[1] for index, row in data.iterrows()]
data = data.loc[data['flux_difference']!=0]
data.sort_values(by='flux_difference', ascending=False)

,1,2,3,4,5,6,7,8,9,10,flux_difference
ME2,1.393249,1.898749,2.404249,2.909749,3.415249,3.920749,4.426249,4.931749,5.437249,5.942749,4.549500
NADTRHD,2.638762,2.986995,3.335227,3.683459,4.031692,4.379924,4.728156,5.076389,5.424621,5.772853,3.134091
MALS,3.396996,3.707393,4.017789,4.328186,4.638583,4.948980,5.259376,5.569773,5.880170,6.190567,2.793571
ICL,3.396996,3.707393,4.017789,4.328186,4.638583,4.948980,5.259376,5.569773,5.880170,6.190567,2.793571
ACLDC,0.292135,0.584270,0.876405,1.168539,1.460674,1.752809,2.044944,2.337079,2.629214,2.921349,2.629214
EX_btd_RR_e,0.292135,0.584270,0.876405,1.168539,1.460674,1.752809,2.044944,2.337079,2.629214,2.921349,2.629214
GLUDy,-2.800815,-2.528102,-2.255389,-1.982676,-1.709962,-1.437249,-1.164536,-0.891823,-0.619109,-0.346396,2.454419
ACLS,0.528293,0.797434,1.066574,1.335714,1.604854,1.873995,2.143135,2.412275,2.681416,2.950556,2.422263
Htex,10.291953,10.536159,10.780366,11.024573,11.268779,11.512986,11.757193,12.001399,12.245606,12.489813,2.197860
PPKr,-13.803374,-13.705676,-13.607978,-13.510281,-13.412583,-13.314885,-13.217187,-13.119490,-13.021792,-12.924094,0.879280


In [56]:
data = data.sort_values(by='flux_difference', ascending=False)
data.to_csv('fseof_analysis_with_POR5.csv')

In [104]:
import plotly.express as px
threshold = data.loc['EX_btd_RR_e', 'flux_difference']

df = data.loc[(data['flux_difference']<=-threshold) | (data['flux_difference']>=threshold) ]
df.pop('flux_difference')
df = df.T
to_remain = ['EX_btd_RR_e', 'FLDR2', 'ACLDC', 'SUCDi', 'POR5', 'FRD2']
df = df[df.columns.intersection(to_remain)]

fig = px.line(df, x="EX_btd_RR_e", y=list(set(df.columns.tolist())-set("EX_btd_RR_e")))

fig.update_layout(    
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text='FSEOF result for isobutanol production', x=0.5),
    xaxis=dict(title='Isobutanol Flux', showline=True, linewidth=3.0, linecolor='black',
               ticks="inside", tickwidth=3.0, tickcolor='black', ticklen=5),
    yaxis=dict(title='Flux of top coupled reactions', showline=True, linewidth=3.0, linecolor='black',
               ticks="inside", tickwidth=3.0, tickcolor='black', ticklen=5),
    font=dict(size=30),
    legend=dict(title='Reactions', font=dict(size=16))
)

fig.update_traces(line=dict(width=3.0))

fig.show()
fig.write_image('FSEOF_WT.svg')

In [60]:
df

,FLDR2,ACLDC,EX_btd_RR_e,ACLS,Htex,BTDD_RR,BTDDtpp,BTDDtex,POR5,NADH17pp,FRD2,SUCDi
1,1.470640,0.326324,0.326324,0.565113,10.264010,-0.326324,-0.326324,-0.326324,-1.470640,16.599254,16.702340,24.817745
2,2.042388,0.652648,0.652648,0.867821,10.514823,-0.652648,-0.652648,-0.652648,-2.042388,15.527991,15.620882,23.482402
3,2.614136,0.978972,0.978972,1.170529,10.765635,-0.978972,-0.978972,-0.978972,-2.614136,14.456728,14.539424,22.147060
4,3.185884,1.305296,1.305296,1.473236,11.016447,-1.305296,-1.305296,-1.305296,-3.185884,13.385465,13.457965,20.811717
5,3.757632,1.631620,1.631620,1.775944,11.267259,-1.631620,-1.631620,-1.631620,-3.757632,12.314202,12.376507,19.476375
6,4.329379,1.957944,1.957944,2.078651,11.518071,-1.957944,-1.957944,-1.957944,-4.329379,11.242939,11.295048,18.141032
7,4.901127,2.284268,2.284268,2.381359,11.768883,-2.284268,-2.284268,-2.284268,-4.901127,10.171676,10.213590,16.805690
8,5.472875,2.610593,2.610593,2.684066,12.019696,-2.610593,-2.610593,-2.610593,-5.472875,9.100413,9.132132,15.470347
9,6.044623,2.936917,2.936917,2.986774,12.270508,-2.936917,-2.936917,-2.936917,-6.044623,8.029150,8.050673,14.135005
10,6.616371,3.263241,3.263241,3.289481,12.521320,-3.263241,-3.263241,-3.263241,-6.616371,6.957887,6.969215,12.799662


In [53]:
alternative_model = final_model.copy()
alternative_model.reactions.POR5.bounds = (0,0)
fseof = FSEOF(alternative_model)
fseof_result = fseof.run(target=alternative_model.reactions.EX_btd_RR_e)

Read LP format model from file /tmp/tmpg9bf61jx.lp
Reading time = 0.02 seconds
: 2032 rows, 5898 columns, 23086 nonzeros


In [55]:
data_alternative = fseof_result.data_frame
data_alternative['flux_difference'] = [row[10]-row[1] for index, row in data_alternative.iterrows()]
data_alternative = data_alternative.loc[data_alternative['flux_difference']!=0]
data_alternative.sort_values(by='flux_difference', ascending=False)

,1,2,3,4,5,6,7,8,9,10,flux_difference
ME2,1.107173,1.640528,2.173883,2.707238,3.240593,3.773948,4.307303,4.840658,5.374013,5.907368,4.800195
NADTRHD,2.352687,2.728774,3.104861,3.480949,3.857036,4.233123,4.609210,4.985298,5.361385,5.737472,3.384785
MALS,3.396996,3.707393,4.017789,4.328186,4.638583,4.948980,5.259376,5.569773,5.880170,6.190567,2.793571
ICL,3.396996,3.707393,4.017789,4.328186,4.638583,4.948980,5.259376,5.569773,5.880170,6.190567,2.793571
ACLDC,0.292135,0.584270,0.876405,1.168539,1.460674,1.752809,2.044944,2.337079,2.629214,2.921349,2.629214
EX_btd_RR_e,0.292135,0.584270,0.876405,1.168539,1.460674,1.752809,2.044944,2.337079,2.629214,2.921349,2.629214
ACLS,0.528293,0.797434,1.066574,1.335714,1.604854,1.873995,2.143135,2.412275,2.681416,2.950556,2.422263
Htex,10.291953,10.536159,10.780366,11.024573,11.268779,11.512986,11.757193,12.001399,12.245606,12.489813,2.197860
PPKr,-13.803374,-13.705676,-13.607978,-13.510281,-13.412583,-13.314885,-13.217187,-13.119490,-13.021792,-12.924094,0.879280
VALTA,-0.271774,-0.245311,-0.218849,-0.192387,-0.165924,-0.139462,-0.112999,-0.086537,-0.060075,-0.033612,0.238162


In [57]:
data_alternative = data_alternative.sort_values(by='flux_difference', ascending=False)
data_alternative.to_csv('fseof_analysis_POR5_knockout.csv')

In [105]:
threshold = data_alternative.loc['EX_btd_RR_e', 'flux_difference']

df = data_alternative.loc[(data_alternative['flux_difference']<=-threshold) | 
                          (data_alternative['flux_difference']>=threshold) ]
df.pop('flux_difference')
df = df.T
to_remain = ['EX_btd_RR_e', 'ACLDC', 'NADTRHD', 'ICL', 'MALS', 'FRD3', 'SUCDi', 'ME2']
df = df[df.columns.intersection(to_remain)]

fig = px.line(df, x="EX_btd_RR_e", y=list(set(df.columns.tolist())-set("EX_btd_RR_e")))

fig.update_layout(    
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text='FSEOF result for isobutanol production knocking out POR5', x=0.5),
    xaxis=dict(title='Isobutanol Flux', showline=True, linewidth=3.0, linecolor='black',
               ticks="inside", tickwidth=3.0, tickcolor='black', ticklen=5),
    yaxis=dict(title='Flux of top coupled reactions', showline=True, linewidth=3.0, linecolor='black',
               ticks="inside", tickwidth=3.0, tickcolor='black', ticklen=5),
    font=dict(size=26),
    legend=dict(title='Reactions', font=dict(size=16), y=0.5)
)

fig.update_traces(line=dict(width=3.0))

fig.show()
fig.write_image('FSEOF_DEL_POR5.svg')